## Fine-tuning BERT (and friends) for multi-label text classification

BERT es un modelo pre-entrenado que captura la información contextual en un corpus extenso. Cuando se adapta a una tarea específica, como clasificación de texto, se agrega una capa lineal al final del modelo pre-entrenado. Esta capa adicional se entrena específicamente para la tarea en cuestión, y su salida es un tensor que contiene puntuaciones para diferentes etiquetas, como categorías de clasificación, para cada ejemplo en un lote de datos.

Estas puntuaciones no están normalizadas, lo que significa que no han sido ajustadas para ser probabilidades directas, pero representan la medida de ajuste o pertinencia del ejemplo a cada una de las etiquetas. Estas puntuaciones se pueden alimentar a una función de activación (como softmax) para normalizarlas y convertirlas en probabilidades asociadas a cada etiqueta, lo que permite tomar decisiones sobre la clasificación del texto.

In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.8 MB/s eta 0:00:00


## 1. IMPORTAMOS LIBRERIAS

In [ ]:
#Manipulacion de datos
import pandas as pd
import numpy as np

#Vizualization
import matplotlib.pyplot as plt
import seaborn as sns

#Preprocessing
from transformers import AutoTokenizer
import torch

#Model
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import EvalPrediction
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.optim import Adam
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, roc_auc_score
from transformers import DistilBertModel, DistilBertTokenizer, AdamW




## 2. CARGAMOS EL DATASET

In [ ]:


df = pd.read_csv("youtoxic_english_1000.csv")

df.T

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
CommentId,Ugg2KwwX0V8-aXgCoAEC,Ugg2s5AzSPioEXgCoAEC,Ugg3dWTOxryFfHgCoAEC,Ugg7Gd006w1MPngCoAEC,Ugg8FfTbbNF8IngCoAEC,Ugg9a6FtoXdxmXgCoAEC,Ugga9KzkNDGvlXgCoAEC,UggBlIXoph7p-3gCoAEC,UggD1aYSn7KOR3gCoAEC,UggGm8a1fu8brngCoAEC,...,Ugh07S5UNzXDWngCoAEC,Ughbo_MZ42mpJ3gCoAEC,Ughd_2zFh-O-93gCoAEC,UghGyVemEvDEqXgCoAEC,UghHMimNd9cYxngCoAEC,Ugi5ADt10EdDz3gCoAEC,Ugifh2DMhBbDkHgCoAEC,Ugj_plbGBjjzYXgCoAEC,Ugj0bah1De8xy3gCoAEC,UgjBJKQSoQMQ6ngCoAEC
VideoId,04kJtp6pVXI,04kJtp6pVXI,04kJtp6pVXI,04kJtp6pVXI,04kJtp6pVXI,04kJtp6pVXI,04kJtp6pVXI,04kJtp6pVXI,04kJtp6pVXI,04kJtp6pVXI,...,XRuCW80L9mA,XRuCW80L9mA,XRuCW80L9mA,XRuCW80L9mA,XRuCW80L9mA,XRuCW80L9mA,XRuCW80L9mA,XRuCW80L9mA,XRuCW80L9mA,XRuCW80L9mA
Text,If only people would just take a step back and...,Law enforcement is not trained to shoot to app...,\nDont you reckon them 'black lives matter' ba...,There are a very large number of people who do...,"The Arab dude is absolutely right, he should h...",here people his facebook is https://www.facebo...,"Check out this you tube post. ""Black man goes ...",I would LOVE to see this pussy go to Staten Is...,I agree with the protestor.,mike browns father was made to say that boooshit,...,"Remember..Michael Brown was just a young, teen...",What point are you trying to make? Is it that...,There are a lot of disgusting people in the co...,"Whites should move-out of Ferguson, let's see ...",I just found this channel and its one of my fa...,I remember that they sent in the national defe...,Stats don`t represent the problem. Race baitin...,The quote from the mother... Wow that hit hard...,this video is so racist,"God, the narrator has such an annoying lisp."
IsToxic,False,True,True,False,False,True,True,True,False,True,...,False,False,False,True,False,False,True,False,False,False
IsAbusive,False,True,True,False,False,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False
IsThreat,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
IsProvocative,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
IsObscene,False,False,True,False,False,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,False
IsHatespeech,False,False,False,False,False,True,True,True,False,False,...,False,False,False,True,False,False,True,False,False,False
IsRacist,False,False,False,False,False,False,True,True,False,False,...,False,False,False,True,False,False,True,False,False,False


## 3. ETIQUETAS

In [ ]:
#Defino las etiquetas
labels = df.iloc[:, 3:15]
labels

,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,False,False,False,False,False,False,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,True,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
995,False,False,False,False,False,False,False,False,False,False,False,False
996,True,False,False,False,False,True,True,False,False,False,False,False
997,False,False,False,False,False,False,False,False,False,False,False,False
998,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
# Codifico las etiquetas.Necesito 1 y 0
labels_codificadas = labels.astype(int)

In [ ]:
labels_codificadas

,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,0,0
996,1,0,0,0,0,1,1,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,0,0


## 4. preprocesamiento

Como los modelos como BERT no esperan texto como entrada directa, sino input_ids, etc., tokenizamos el texto usando el tokenizador.

input_ids se refiere a una secuencia de números enteros que representan las palabras o tokens en un texto. Cada palabra o token se asigna a un identificador único, y BERT requiere que cada una de estas palabras o tokens se convierta en un número entero específico antes de ser procesada por el modelo.

Además,tenemos que proporcionar etiquetas al modelo. Para la clasificación de texto multietiqueta, se trata de una matriz de forma (batch_size, num_labels). También es importante: esto debe ser un tensor de flotantes en lugar de enteros, de lo contrario PyTorch' BCEWithLogitsLoss (que utilizará el modelo) se quejará.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
    text = examples["Text"]
    encoding = tokenizer(text.tolist(), padding="max_length", truncation=True, max_length=128, return_tensors="pt", return_token_type_ids=True )

    labels_batch = examples.iloc[:, 3:15]  # Selecciona las columnas que representan las etiquetas
    labels_matrix = labels_batch.to_numpy(dtype=int)  # Convierte directamente a un array de numpy con tipo int

    encoding["labels"] = labels_matrix

    return encoding, labels_matrix

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
datos_procesados, labels_codificadas = preprocess_data(df)

In [ ]:
# Convertir los datos a tensores de PyTorch
input_tensors = torch.tensor(datos_procesados["input_ids"])
attention_masks = torch.tensor(datos_procesados["attention_mask"])
labels_tensors = torch.tensor(labels_codificadas)

input_tensors: Contiene los IDs de los tokens de entrada después de la tokenización, representando las secuencias de texto.

attention_masks: Representa las máscaras de atención para indicar a BERT qué tokens deben ser atendidos y cuáles no.

labels_tensors: Contiene las etiquetas codificadas para la tarea que estás abordando, como por ejemplo, la clasificación multietiqueta.

En BERT, se utilizan dos tipos de máscaras:

Máscara de Relleno (Padding Mask): Indica a BERT qué tokens son reales y cuáles son relleno. Los tokens de relleno suelen ser aquellos añadidos para hacer que todas las secuencias tengan la misma longitud dentro de un lote.

Máscara de Atención (Attention Mask): Indica a BERT qué tokens deben ser atendidos y cuáles no. Para tokens reales, la máscara es 1, indicando que se deben atender; para tokens de relleno, la máscara es 0, indicando que no se debe prestar atención a ellos.

## 5. DIVIDIMOS LOS DATOS EN TRAIN-TEST

In [ ]:
# Convertir los tensores a arrays de NumPy
X = input_tensors.numpy()
M = attention_masks.numpy()
y = labels_tensors.numpy()

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, M_train, M_test, y_train, y_test = train_test_split(X, M, y, test_size=0.2, random_state=42)

# Revisar las formas de los conjuntos para asegurarte de que la división sea correcta
print("Forma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de M_train:", M_train.shape)  # Agregado para verificar la forma de M_train
print("Forma de M_test:", M_test.shape)    # Agregado para verificar la forma de M_test
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)

Forma de X_train: (800, 128)
Forma de X_test: (200, 128)
Forma de M_train: (800, 128)
Forma de M_test: (200, 128)
Forma de y_train: (800, 12)
Forma de y_test: (200, 12)


## 6. ENTRENAMIENTO DEL MODELO

Aquí definimos un modelo que incluye una base preentrenada (es decir, se cargan los pesos de bert-base-uncased), con una cabeza de clasificación inicializada aleatoriamente (capa lineal) en la parte superior. Uno debe afinar esta cabeza, junto con la base pre-entrenada en un conjunto de datos etiquetados.

Establecemos el tipo_problema como "multi_label_classification", ya que así nos aseguramos de que se utiliza la función de pérdida adecuada (es decir, BCEWithLogitsLoss). También nos aseguramos de que la capa de salida tiene len(labels) neuronas de salida.

In [ ]:
#MODELO BERT
#Defino el modelo
model_name = "bert-base-uncased"
num_labels = 12

# Inicialización del modelo y el tokenizador para clasificación de secuencias
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    problem_type="multi_label_classification",
    num_labels=num_labels
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Verificación de la arquitectura del modelo
print(model)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
#LO ENTRENO

# Define la función de pérdida (criterio)
criterion = nn.BCEWithLogitsLoss()  # Para clasificación multi-etiqueta

# Define el optimizador
optimizer = Adam(model.parameters(), lr=1e-5)  # Ajusta la tasa de aprendizaje según sea necesario

# Número de épocas para entrenar el modelo
num_epochs = 3  # Ajusta este número según lo que requiera tu modelo

# Entrenamiento del modelo

model.train()  # Modo de entrenamiento
for epoch in range(num_epochs):
    running_loss = 0.0
    running_precision = 0.0
    running_recall = 0.0
    running_f1 = 0.0
    running_accuracy = 0.0
    running_confusion_matrix = None

    for i in range(len(X_train)):
        optimizer.zero_grad()

        inputs = torch.tensor(X_train[i]).unsqueeze(0)
        masks = torch.tensor(M_train[i]).unsqueeze(0)
        labels = torch.tensor(y_train[i]).unsqueeze(0).float()

        outputs = model(inputs, attention_mask=masks, labels=labels)
        loss = criterion(outputs.logits, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} - Loss: {running_loss/len(X_train)}")






Epoch 1 - Loss: 0.3127327902521938
Epoch 2 - Loss: 0.22193441233132036
Epoch 3 - Loss: 0.16110881707863883


In [24]:
# Evaluación del modelo en conjunto de entrenamiento
model.eval()

predictions_train = []

for i in range(len(X_train)):
    inputs = torch.tensor(X_train[i]).unsqueeze(0)  # Asegura las dimensiones
    masks = torch.tensor(M_train[i]).unsqueeze(0)

    outputs = model(input_ids=inputs, attention_mask=masks)
    predicted_labels = torch.sigmoid(outputs.logits) > 0.5  # Threshold de 0.5 para predicciones

    predictions_train.append(predicted_labels.detach().numpy())

predictions_train = np.vstack(predictions_train)

# Cálculo de métricas en conjunto de entrenamiento
accuracy_train = accuracy_score(y_train, predictions_train)

print(f"Accuracy en conjunto de entrenamiento: {accuracy_train}")

Accuracy en conjunto de entrenamiento: 0.71875


## 7. EVALUACIÓN DEL MODELO

In [25]:
model.eval()

with torch.no_grad():
    predictions = []

    for i in range(len(X_test)):
        inputs = torch.tensor(X_test[i]).unsqueeze(0)
        masks = torch.tensor(M_test[i]).unsqueeze(0)
        labels = torch.tensor(y_test[i]).unsqueeze(0)

        outputs = model(inputs, attention_mask=masks)
        predicted_labels = torch.sigmoid(outputs.logits) > 0.5

        predictions.append(predicted_labels.detach().numpy())

    predictions = np.vstack(predictions)

    # Cálculo de métricas
    precision = precision_score(y_test, predictions, average='micro')
    recall = recall_score(y_test, predictions, average='micro')
    f1 = f1_score(y_test, predictions, average='micro')
    accuracy_test = accuracy_score(y_test, predictions)

    print(f"Epoch {epoch+1} - Precision: {precision}, Recall: {recall}, F1-score: {f1}, Accuracy: {accuracy}")

Epoch 3 - Precision: 0.8080808080808081, Recall: 0.4984423676012461, F1-score: 0.6165703275529866, Accuracy: 0.5


In [26]:

overfitting = accuracy_train - accuracy_test
print(f'Overfitting: {overfitting}')

Overfitting: 0.21875


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8d0efb33-cf13-434e-b2e8-97275399daa5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>